In [ ]:
# Step 1: Install dependency
!pip install ffmpeg-python

# Step 2: Clone the Wav2Lip repository
!git clone https://github.com/justinjohn0306/Wav2Lip

# Step 3: Download pretrained model
import requests
url = "https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA"
response = requests.get(url)

with open("Wav2Lip/checkpoints/wav2lip_gan.pth", "wb") as f:
    f.write(response.content)
    
# Step 4: Install the required dependencies for Wav2Lip
!cd Wav2Lip && pip install -r requirements.txt
!pip install pyaudio


# Step 5: Download pretrained model for face detection
url = "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth"
response = requests.get(url)

with open("Wav2Lip/face_detection/detection/sfd/s3fd.pth", "wb") as f:
    f.write(response.content)

In [ ]:
import os
import subprocess
from urllib import parse as urlparse

# Step 1: Install yt-dlp
subprocess.run(['pip', 'install', 'yt-dlp'])

# Step 2: Define YouTube URL and Video ID
YOUTUBE_URL = 'https://www.youtube.com/watch?v=vAnWYLTdvfY'
url_data = urlparse.urlparse(YOUTUBE_URL)
query = urlparse.parse_qs(url_data.query)
YOUTUBE_ID = query["v"][0]

# Remove previous input video
if os.path.isfile('input_vid.mp4'):
    os.remove('input_vid.mp4')

# Trim video (start, end) seconds
start = 35
end = 62
interval = end - start

# Step 3: Download and trim the YouTube video
subprocess.run(['yt-dlp', '-f', 'bestvideo[ext=mp4]', '--output', "youtube.%(ext)s", f'https://www.youtube.com/watch?v={YOUTUBE_ID}'])

# Cut the video using FFmpeg
subprocess.run(['ffmpeg', '-y', '-i', 'youtube.mp4', '-ss', str(start), '-t', str(interval), '-async', '1', 'input_vid.mp4'])

# Display video.
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""<video width=600 controls><source src="{data_url}"></video>""")

# Preview the trimmed video
show_video('input_vid.mp4')

In [ ]:
import os
from IPython.display import Audio
from IPython.core.display import display

upload_method = 'Path'  # Change this to 'Record' or 'Path'

# Remove previous input audio
if os.path.isfile('input_audio.wav'):
    os.remove('input_audio.wav')

def display_audio():
    display(Audio('input_audio.wav'))

if upload_method == 'Record':
    import pyaudio
    import wave

    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "input_audio.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    display_audio()

elif upload_method == 'Path':
    # Add the full path to your audio
    PATH_TO_YOUR_AUDIO = 'C:/Users/justi/OneDrive/Desktop/wav2lip/Wav2Lip/input_audio.wav'

    # Load audio with specified sampling rate
    import librosa
    audio, sr = librosa.load(PATH_TO_YOUR_AUDIO, sr=None)

    # Save audio with specified sampling rate
    import soundfile as sf
    sf.write('input_audio.wav', audio, sr, format='wav')

    display_audio()

In [ ]:
# Define the parameters for the Wav2Lip model
pad_top = 0
pad_bottom = 10
pad_left = 0
pad_right = 0
rescaleFactor = 1
nosmooth = False

# Set the path to the Wav2Lip model and input files
checkpoint_path = "checkpoints/wav2lip_gan.pth"
input_face = "input_vid.mp4"
input_audio = "input_audio.wav"

# Run the Wav2Lip model
!cd Wav2Lip && python inference.py --checkpoint_path {checkpoint_path} --face {input_face} --audio {input_audio} --pads {pad_top} {pad_bottom} {pad_left} {pad_right} --resize_factor {rescaleFactor} {"--nosmooth" if nosmooth else ""}

# Preview the output video
print("Final Video Preview")
print("Find the output video at", 'Wav2Lip/results/result_voice.mp4')
show_video('Wav2Lip/results/result_voice.mp4')